In [1]:
!pip install selenium
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import MoveTargetOutOfBoundsException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import os
import pandas as pd

In [2]:
def click_by_time(driver,xpath,maxTime):
    t=0
    while t<=maxTime:
        if driver.find_element(By.XPATH, xpath)!=None:  
            driver.find_element(By.XPATH, xpath).click()
            break
        time.sleep(1)
        t+=1  

In [3]:

# First Round
# keywords = ['chinese privilege', 'malay privilege', 'indian privilege', 'majority privilege', 'meritocracy',
#             'chinese majority', 'underprivileged family', 'subsidies', 'subsidy', 'mendaki', 'CDAC', 'malay subject',
#             'student', 'Education gap', 'Education Background', 'SINDA', 'University-Educated Malay', 'ceca', 'influx',
#             'speak mandarin', 'chinese speaking', 'Malay coworker', 'Denied Jobs', 'Career success', 'career choices',
#             'Chinese Speakers', 'Indian IT', 'rental', 'rent', 'Living in HDB', 'Household incomes', 'HDB',
#             'colored people', 'colored kids', 'skin black', 'skin color', 'black skin', 'yellow skin', 'brown skin',
#             'darked-skinned', 'dark skin', 'tanned skin', 'fair skin', 'faired skin', 'keling kia', 'local chinese',
#             'low ses', 'Victim Card', 'High SES', 'Chinese speaking', 'Chinese language', 'Speaking Mandarin',
#             'Speak Malay', 'Speak Indian', 'Speak Tamil', 'sinkie chinese', 'chinese culture', 'chinese paper',
#             'chinese majority', 'chinese in singapore', 'Malay Culture', 'Indian Culture', 'CMIO', 'CMIO Model',
#             'Malay Community', 'Lazy Malay', 'Indian IT', 'Stereotypes minorities', 'Poor Chinese', 'rich Malay',
#             'rich Indian', 'rich Chinese', 'Poor Malay', 'Poor Indian', 'lifestyle preference']


# Second Round
keywords = ['middle income', 'high income', 'low income', 'income inequality', 'disposable income', 'household annual income',
            'wealth tax', 'unemployment insurance', 'meritocracy', 'occupation mobility', 'social mobility', 'occupational prestige',
            'parental occupation', 'professional conversion programme', 'skillsfuture', 'elite education', 'education subsidies', 'Mendaki',
            'CDAC', 'less educated', 'educational attainment', 'achievement gap', 'access to education', 'health insurance', 'medical expenses',
            'medical resources', 'Medisave', 'MediShield Life', 'Community Health Assit Scheme', 'access to healthcare', 'upper class',
            'lower class', 'middle class', 'central provident fund', 'CPF', 'low income group', 'middle income group', 'high income group', 'poverty']


print(keywords)
        
search_root_url = 'https://www.bing.com/search?q=site%3Aforums.hardwarezone.com.sg%2F+'


['middle income', 'high income', 'low income', 'income inequality', 'disposable income', 'household annual income', 'wealth tax', 'unemployment insurance', 'meritocracy', 'occupation mobility', 'social mobility', 'occupational prestige', 'parental occupation', 'professional conversion programme', 'skillsfuture', 'elite education', 'education subsidies', 'Mendaki', 'CDAC', 'less educated', 'educational attainment', 'achievement gap', 'access to education', 'health insurance', 'medical expenses', 'medical resources', 'Medisave', 'MediShield Life', 'Community Health Assit Scheme', 'access to healthcare', 'upper class', 'lower class', 'middle class', 'central provident fund', 'CPF', 'low income group', 'middle income group', 'high income group', 'poverty']


In [4]:

def search_by_key_word(search_term):
    search_url = search_root_url + search_term
    company_urls = []

    driver = webdriver.Chrome() 
    driver.get(search_url)
    click_by_time(driver,'//*[@id="sb_form_go"]',10) 

    count = 0
    records = 0
    no_valid_result_count = 0
    prev_valid_result = -1
    while(no_valid_result_count < 4):
        count += 1
        time.sleep(5)

        if len(company_urls) == prev_valid_result:
            no_valid_result_count += 1
        else:
            no_valid_result_count = 0

        prev_valid_result = len(company_urls)

        titles = driver.find_elements(By.CLASS_NAME, 'b_algo') 
        for title in titles:
            result = title.find_element(By.TAG_NAME, 'a') 

            spans = title.find_elements(By.TAG_NAME, 'a')
            for span in spans:
                href = span.get_attribute('href')
                if 'threads' in str(href) :
                    print(str(href))
                    company_urls.append(str(href)) 


        company_urls = list(set(company_urls)) 
        print(count,'page','total',len(company_urls),' valid results for term ' + search_term)
        try:
            pagebar = driver.find_element(By.CLASS_NAME, 'b_pag') 
            next_page = pagebar.find_elements(By.TAG_NAME, 'a')[-1]
            next_page.click()
        except:
            print('Search bar error for term ' + search_term)
                
    output_dict = {'URL': company_urls}
    output_df = pd.DataFrame(output_dict, columns = output_dict.keys())
    output_df['Search_Term'] = search_term
    driver.close()
    time.sleep(5)

    return output_df

In [5]:
df = pd.DataFrame()

frames = []
for key_word in keywords:
    try:
        tmp = search_by_key_word(key_word)
        frames.append(tmp)
    except Exception as e:
        print("An exception occurred for " + key_word + " error: " + str(e))
resultDf = pd.concat(frames)
resultDf.to_csv('hardware_zone_raw_data_without_detail.csv')

https://forums.hardwarezone.com.sg/threads/how-much-would-you-consider-middle-income-3k-enough.6710002/
https://forums.hardwarezone.com.sg/threads/claims-that-middle-income-singaporeans-are-overtaxed-an-outright-falsehood-dpm-wong.6876381/
https://forums.hardwarezone.com.sg/threads/what-is-middle-class-upper-middle-class-income.6809953/
https://forums.hardwarezone.com.sg/threads/the-taxman-has-launched-an-audit-of-property-investors-who-have-used-a-loophole-99-to-1-sales-contract-to-dodge-paying-the-additional-buyers-stamp.6888745/page-43
https://forums.hardwarezone.com.sg/threads/wah-income-tax-merry-pain.6887835/page-4
https://forums.hardwarezone.com.sg/threads/the-taxman-has-launched-an-audit-of-property-investors-who-have-used-a-loophole-99-to-1-sales-contract-to-dodge-paying-the-additional-buyers-stamp.6888745/page-44
https://forums.hardwarezone.com.sg/threads/can-anyone-tell-me-why-is-sg-getting-worse-as-years-passed.6879805/
https://forums.hardwarezone.com.sg/threads/discussion-

Search bar error for term middle income
https://forums.hardwarezone.com.sg/threads/can-i-ask-hdb-to-change-my-hle-from-family-application-to-single.6886459/
https://forums.hardwarezone.com.sg/threads/tightening-access-to-lower-skilled-foreign-workers-more-likely-to-improve-job-outcomes-for-locals-mom.6877276/page-4
https://forums.hardwarezone.com.sg/threads/actually-whats-your-reason-for-staying-in-sg.6875736/
https://forums.hardwarezone.com.sg/threads/do-you-think-you-can-ever-make-enough-money-to-enjoy-life.6887210/page-4
https://forums.hardwarezone.com.sg/threads/recently-i-found-out-a-lot-of-malaysian-chinese-only-earns-2-k-or-2-k-plus.6882556/page-10
https://forums.hardwarezone.com.sg/threads/actually-whats-your-reason-for-staying-in-sg.6875736/page-2
https://forums.hardwarezone.com.sg/threads/how-to-not-work-and-get-money.6877918/page-2
https://forums.hardwarezone.com.sg/threads/household-income-in-singapore-do-you-belong-to-top-16.6873072/page-4
https://forums.hardwarezone.com.s

https://forums.hardwarezone.com.sg/threads/are-high-networth-individuals-with-zero-income-living-in-hdb-eligible-for-chas-card.6863147/page-3
https://forums.hardwarezone.com.sg/threads/high-income-low-wealth-over-half-of-adults-in-singapore-dont-watch-their-spending.6872117/
https://forums.hardwarezone.com.sg/threads/if-you-work-in-sg-and-live-in-jb-you-are-a-t20-high-income-group.6887514/
https://forums.hardwarezone.com.sg/threads/wah-income-tax-merry-pain.6887835/page-2
1 page total 10  valid results for term high income
https://forums.hardwarezone.com.sg/threads/income-tax-filing-2023.6879395/page-9
https://forums.hardwarezone.com.sg/threads/very-shag-who-need-to-pay-income-tax-5k.6882209/
https://forums.hardwarezone.com.sg/threads/wah-income-tax-merry-pain.6887835/page-2
https://forums.hardwarezone.com.sg/threads/income-tax-filing-2023.6879395/page-8
https://forums.hardwarezone.com.sg/threads/very-shag-who-need-to-pay-income-tax-5k.6882209/page-2
https://forums.hardwarezone.com.sg/

https://forums.hardwarezone.com.sg/threads/hdb-to-pilot-new-type-of-public-rental-housing-with-own-room-shared-facilities-for-low-income-singles.6881898/page-18
https://forums.hardwarezone.com.sg/threads/commentary-singapore-residents-must-save-for-retirement-today-to-continue-enjoying-low-income-taxes.6875787/
https://forums.hardwarezone.com.sg/threads/hdb-is-affordable-not-sarcasm.6870477/page-15
https://forums.hardwarezone.com.sg/threads/home-kitchens-to-disrupt-inflation.6880899/
https://forums.hardwarezone.com.sg/threads/why-diploma-and-degree-courses-cost-so-much.6886584/page-2
1 page total 10  valid results for term low income
https://forums.hardwarezone.com.sg/threads/hdb-to-pilot-new-type-of-public-rental-housing-with-own-room-shared-facilities-for-low-income-singles.6881898/
https://forums.hardwarezone.com.sg/threads/hdb-to-pilot-new-type-of-public-rental-housing-with-own-room-shared-facilities-for-low-income-singles.6881898/page-10
https://forums.hardwarezone.com.sg/threads/

https://forums.hardwarezone.com.sg/threads/negative-people-club.6491048/page-2334
7 page total 36  valid results for term low income
https://forums.hardwarezone.com.sg/threads/when-govt-says-household-median-income-9k-people-ask-is-it-true-but-yet-some-want-up-income-limit-for-cpf-topup-4-000-income-limit-too-low.6887247/
8 page total 36  valid results for term low income
https://forums.hardwarezone.com.sg/threads/when-govt-says-household-median-income-9k-people-ask-is-it-true-but-yet-some-want-up-income-limit-for-cpf-topup-4-000-income-limit-too-low.6887247/
9 page total 36  valid results for term low income
https://forums.hardwarezone.com.sg/threads/when-govt-says-household-median-income-9k-people-ask-is-it-true-but-yet-some-want-up-income-limit-for-cpf-topup-4-000-income-limit-too-low.6887247/
10 page total 36  valid results for term low income
https://forums.hardwarezone.com.sg/threads/when-govt-says-household-median-income-9k-people-ask-is-it-true-but-yet-some-want-up-income-limit

...

Search bar error for term middle income group
https://forums.hardwarezone.com.sg/threads/very-shag-who-need-to-pay-income-tax-5k.6882209/page-4
https://forums.hardwarezone.com.sg/threads/what-is-considered-middle-class-in-sg.6872918/page-6
https://forums.hardwarezone.com.sg/threads/general-s-reits-discussion-thread.4313167/page-294
https://forums.hardwarezone.com.sg/threads/highest-income-group-could-be-most-affected-by-high-inflation-analysts.6871030/page-2
https://forums.hardwarezone.com.sg/threads/household-income-in-singapore-do-you-belong-to-top-16.6873072/page-4
https://forums.hardwarezone.com.sg/threads/how-to-not-work-and-get-money.6877918/page-2
https://forums.hardwarezone.com.sg/threads/negative-people-club.6491048/page-1992
https://forums.hardwarezone.com.sg/threads/reality-of-being-a-middle-aged-single-in-sg.6881174/page-21
https://forums.hardwarezone.com.sg/threads/its-increasingly-harder-to-live-a-life-in-sg.6884862/page-4
https://forums.hardwarezone.com.sg/threads/why-do

https://forums.hardwarezone.com.sg/threads/desmond-lee-says-singaporeans-accept-high-price-bto-flats-in-mature-estates.6887271/page-2
https://forums.hardwarezone.com.sg/threads/which-direction-will-property-prices-go.6382009/page-743
https://forums.hardwarezone.com.sg/threads/official-crypto-currency-chit-chat-group-part-6.6479363/page-3521
16 page total 41  valid results for term middle income group
https://forums.hardwarezone.com.sg/threads/official-crypto-currency-chit-chat-group-part-6.6479363/page-3521
17 page total 41  valid results for term middle income group
https://forums.hardwarezone.com.sg/threads/highest-income-group-could-be-most-affected-by-high-inflation-analysts.6871030/page-2
https://forums.hardwarezone.com.sg/threads/reality-of-being-a-middle-aged-single-in-sg.6881174/page-21
https://forums.hardwarezone.com.sg/threads/pricing-new-flats-even-lower-would-only-increase-the-windfall-gain-that-successful-buyers-enjoy.6869842/page-8
https://forums.hardwarezone.com.sg/threa

https://forums.hardwarezone.com.sg/threads/how-much-you-require-to-retire.6886845/page-4
https://forums.hardwarezone.com.sg/threads/sheng-siong-official-sgx-ov8.4053726/page-162
https://forums.hardwarezone.com.sg/threads/very-shag-who-need-to-pay-income-tax-5k.6882209/page-5
6 page total 21  valid results for term high income group
https://forums.hardwarezone.com.sg/threads/if-you-work-in-sg-and-live-in-jb-you-are-a-t20-high-income-group.6887514/
https://forums.hardwarezone.com.sg/threads/income-tax-filing-2023.6879395/page-10
https://forums.hardwarezone.com.sg/threads/incum-tax-filing-season-ish-it-possible-to-hit-80k-relief.6887899/
https://forums.hardwarezone.com.sg/threads/would-u-believe-someone-to-be-a-high-earner-if-dont-even-own-a-car.6887251/page-4
https://forums.hardwarezone.com.sg/threads/very-shag-who-need-to-pay-income-tax-5k.6882209/page-4
https://forums.hardwarezone.com.sg/threads/wah-income-tax-merry-pain.6887835/page-3
https://forums.hardwarezone.com.sg/threads/serious

https://forums.hardwarezone.com.sg/threads/as-parents-how-will-u-feel-if-ur-daughter-becomr-of-creatir-like-ms-puiyi.6888172/page-2
https://forums.hardwarezone.com.sg/threads/every-income-tax-season-do-iras-check-on-influencers-youtubers-who-make-big-money.6889246/
https://forums.hardwarezone.com.sg/threads/why-are-sinkies-not-kicking-up-a-fuss-like-the-french-do.6887956/page-4
https://forums.hardwarezone.com.sg/threads/circuit-rd-hawker-stall-selling-handmade-australian-style-minced-pies-closing-on-apr-30-2023-due-to-high-rent-and-unable-to-break-even.6889334/page-4
13 page total 31  valid results for term high income group
https://forums.hardwarezone.com.sg/threads/if-you-work-in-sg-and-live-in-jb-you-are-a-t20-high-income-group.6887514/
https://forums.hardwarezone.com.sg/threads/wah-income-tax-merry-pain.6887835/page-2
https://forums.hardwarezone.com.sg/threads/income-tax-filing-2023.6879395/page-10
https://forums.hardwarezone.com.sg/threads/would-u-believe-someone-to-be-a-high-earn

https://forums.hardwarezone.com.sg/threads/young-gig-workers-need-educating-on-harsh-realities-of-platform-work.6887900/page-3
https://forums.hardwarezone.com.sg/threads/now-is-it-very-common-to-pay-7-8-for-cai-png-outside.6887998/page-4
https://forums.hardwarezone.com.sg/threads/pla-want-to-dominate-the-world.6888564/latest
https://forums.hardwarezone.com.sg/threads/melbournes-singapore-community-at-a-glance.6786020/page-20
https://forums.hardwarezone.com.sg/threads/singapores-total-fertility-rate-drops-to-historic-low-of-1-05.6876304/page-9
https://forums.hardwarezone.com.sg/threads/serious-question-is-annual-income-above-100k-common.6874077/page-14
https://forums.hardwarezone.com.sg/threads/y-some-ppl-like-to-use-pirated-stuff.6887233/
https://forums.hardwarezone.com.sg/threads/how-the-hell-is-sg-the-happiest-country-in-asia.6886027/page-2
https://forums.hardwarezone.com.sg/threads/charlie-munger-on-lee-kuan-yew-and-singapores-success.6888656/page-3
https://forums.hardwarezone.com.s

In [6]:
print('End of scrapping')

End of scrapping
